In [1]:
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, RadioGroup, Slider
from bokeh.io import show, output_notebook
from bokeh import palettes
output_notebook()

Loading BokehJS ...

In [2]:
import random
from functools import partial
import numpy as np
import pandas as pd
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import fitstools
from pathlib import PurePath
import glob

In [3]:
# Get some data into a dataframe
x1 = list(range(0, 10))
y1 = random.sample(range(0, 10), 10)
# Prepare an output dataframe that will contain the labels (initially, there is none)
df = pd.DataFrame({'x':x1, 'y':y1, 'label': 'no label'})
df.set_index('x', inplace=True)

In [53]:
# Data prepped by COR2_tracking_prep.ipynb
datadir = PurePath('C:\Data\STEREO\L7tum\prep_fits')
outputdir = PurePath('C:\Data\STEREO\L7tum\\figures')
datafiles = sorted(glob.glob(str(PurePath(datadir, '*.fits'))))
nfiles = len(datafiles)
nsamples = 80
fov = np.s_[0:659, 0:600]
samples = [np.abs(fitstools.fitsread(datafiles[i], cube=False, astropy=True))[fov] for i in range(10)]

In [65]:
# Define the app. The "doc" argument is the container of your app. We add to it the graphs and GUI elements
def bkapp(doc):
    # Define the tools that will appear in the figure toolbar. 
    # The selection tool is necessary, e.g. `box_select`.
    tools = ['box_select', 'hover', 'reset']
    TOOLTIPS = [
        ('index', "$index"),
        ("x", "$x"),
        ("y", "$y"),
        ("value", "@image"),
    ]
    # Create the figure axis
    p = figure(x_axis_label='x',
               y_axis_label='X [px]',
               title="Test displaying image series",
               tooltips=TOOLTIPS,
               tools=tools)
    p.x_range.range_padding = p.y_range.range_padding = 0
    # Display the image
    p.image(image=[samples[0]], x=0, y=0, dw=600, dh=659, palette=palettes.Greys256)

    myslider = Slider(title="amplitude", value=0, start=0, end=10, step=1)
    
    def callback2(attr, old, new):
        frame_nb = myslider.value
        p.image(image=[samples[frame_nb]], x=0, y=0, dw=600, dh=659, palette=palettes.Greys256)

    # scatter.data_source.selected.on_change('indices', callback1)
    myslider.on_change('value_throttled', callback2)
    
    # Add our GUI and graph in a column layout
    doc.add_root(column(myslider, p))

In [66]:
show(bkapp)

In [51]:
# Define the app. The "doc" argument is the container of your app. We add to it the graphs and GUI elements
def bkapp(doc):
    N1 = 600
    N2 = 659
    x = np.linspace(0, 10, N1)
    y = np.linspace(0, 10, N2)
    xx, yy = np.meshgrid(x, y)
    # Random periodic data with noise
    noise_data = [np.sin(xx)*np.cos(yy) + np.random.rand(N2,N1) for _ in range(10)]
    
    tools = ['box_select', 'hover', 'reset']
    myslider = Slider(title="amplitude", value=0, start=0, end=10, step=1)
    # Create the figure axis
    p = figure(x_axis_label='X [px]',
                  y_axis_label='Y [px]',
                  title="Test displaying image series",
                  tools=tools)
    
    p.x_range.range_padding = p.y_range.range_padding = 0
    # Display the image
    p.image(image=[noise_data[myslider.value]], x=0, y=0, dw=N1, dh=N2, palette=palettes.Greys256)
   
        
    def callback(attr, old, new):
        p.image(image=[noise_data[myslider.value]], x=0, y=0, dw=N1, dh=N2, palette=palettes.Greys256)
        print(myslider.value)

    # scatter.data_source.selected.on_change('indices', callback1)
    myslider.on_change('value', callback)
    
    # Add our GUI and graph in a column layout
    doc.add_root(column(myslider, p))

In [52]:
show(bkapp)

1
2
3
